# Building a webserver with Flask

![xkcd](http://imgs.xkcd.com/comics/countdown.png)


## What is Flask?

Flask is a **micro web application framework**.

You can use Flask to build dynamic websites,
for example a user interface to your Python program.

Some Flask features:
* Development server and debugger
* Integrated support for unit testing
* No database abstraction layer, no form validation or similar advanced features 

### Installation

```bash
conda install flask
``` 

## Hello world in Flask

In [1]:
# File: hello_world.py
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Run the flask server with:
```bash
python hello_world.py
   * Running on http://localhost:5000/
```            

Then open http://localhost:5000/ in your web browser. To stop the server, hit `Control-C`.


## What happened?

```python
from flask import Flask
app = Flask(__name__)
```
The instance of the `Flask` class is our web application.

The first argument is needed so that Flask knows where to look for templates, static files, and so on.  If you are using a single module simply use `__name__`. 

```python
@app.route("/")
def hello():
    return "Hello World!"
```

We then use the route() decorator to tell Flask what URL should trigger our function. By default, Flask answers to GET requests, and the return value of the function is the answer of the `GET` request.

```python
if __name__ == "__main__":
    app.run()
```

Finally we use the `run()` function to run the local server with our application. 

**Note**: You need to restart the sever when changing your code.

## Debug mode
You can use 
```python
app.run(debug=True)
``` 
to see more detailed error output. 

In debug mode the server will reload itself on code changes. 


## Making the server available in your network

If you want your server to be accessible from your entire network, you need to launch the server with:
```python
app.run(host='0.0.0.0')
```

**Important**: This allows *anyone* in your network to access your webbrowser, which might be a severe security riks!

## Adding more URLs

We can serve additional URLs by adding new functions with the route decorator:

In [2]:
users = {"richard": "Richard Lee",
         "john": "John Smith"}

@app.route('/user')
def show_user_overview():
    users_str = "<br>".join(users.values())
    return '<h1>Our users</h1><br>{}'.format(users_str)
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


Matches http://localhost:5000/user

In [3]:
@app.route('/user/<username>')
def show_user_profile(username):
    # show the user profile for that user
    return 'Welcome {}'.format(username)
app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [30/Jan/2017 23:21:23] "GET /user/NAME HTTP/1.1" 200 -
127.0.0.1 - - [30/Jan/2017 23:21:27] "GET /user/2.14 HTTP/1.1" 200 -


Matches http://localhost:5000/user/NAME for any `NAME`

## Python challenge

1. Write a Python web server that converts between Fahrenheit and Celcius. It should match URLs following
    * http://localhost:5000/fahrenheit_to_celcius/75 
      The resulting webpage should display "75 Fahrenheit is 23.89 Celcius"
    * http://localhost:5000/celcius_to_fahrenheit/23.89
      The resulting webpage should display "23.89 Fahrenheit is 75 Fahrenheit"
    
2. Add error handling of invalid inputs, that is display a error message in the webbrowser if an invalid number is entered.
   
3. **Bonus**: Add more conversion functions.


## Next we need to learn some HTML

See the slides [here](http://nbviewer.jupyter.org/github/UiO-INF3331/UiO-INF3331.github.io/blob/crash-course/lectures/web_programming/Introduction%20to%20HTML.ipynb).

## Templates

So far our webserver only served simple textstrings, but no HTML documents. 

One solution would be to define the entire HTML string in the URL handler, e.g.:

```python
@app.route('/login')
def login():
    return '''
<html>
<header><title>The title</title></header>
<body>
Hello world
</body>
</html>    
    '''
```

However, Flask comes with a templating system that makes our life a lot easier:

In [ ]:
from flask import render_template

@app.route('/post/')
@app.route('/post/<name>')
def post(name=None):
    return render_template('post.html', name=name)

Flask will look for templates in the `templates` folder.

```html
<!-- ./templates/post.html -->

<!doctype html>
<title>Hello from Flask</title>
{% if name %}
  <h1>Displaying blog post {{ name }}!</h1>
{% else %}
  <h1>No post name given!</h1>
{% endif %}
```

In [ ]:
app.run()

### Python challenge

1. Change your Fahrenheit/Celcius converter website such that it uses template files.
   
   **Unsure how to get started?**: Download the above [demo server](https://github.com/UiO-INF3331/UiO-INF3331.github.io/raw/crash-course/lectures/web_programming/web_programming_template.zip) with a working template.

2. **Bonus**: Style the template files with some HTML.

## HTML form

Using the template, we can now create a HTML form with a `POST` request

```html
<!-- ./templates/login.html -->

<!doctype html>
<title>Login</title>

{% if error %}
<p style="color:red">{{ error }}</p>
{% endif %}

<form action="handle_login" method="POST">
    Username:
    <br>
    <input type="text" name="username">
    <br>
    Password:
    <br>
    <input type="password" name="password">
    <br>
    <input type="submit" value="Submit">
</form>
```


In [ ]:
@app.route('/login')
def login():
    return render_template('login.html')

In [ ]:
app.run()

## Handling the `POST` request.

The form above sends a `POST` request to the `handle_login` URL. 

We can use 
```python
@app.route('/handle_login', methods=['POST'])     
```
to create a new Flask handler that accepts `POST` requests.

We can then use the 
```python
flask.request
```
module to access the data in the `POST` request (here the username and password that the user provided in the form).

In [ ]:
from flask import request
                                                                                                                                                                                                                                               
@app.route('/handle_login', methods=['POST'])                                                                                                                                                                                                  
def handle_login():           
    
    assert request.method == 'POST'   # Check that we are really in a POST request
    
    # Acces the form data:
    username = request.form["username"]
    password = request.form["password"]
    
    if username == "simon" and password == "safe":                                                                                                                                                             
        return "You are logged in Simon"                                                                                                                                                                                                       
    else:                                                                                                                                                                                                                                      
        error = "Invalid credentials"                                                                                                                                                                                                          
        return render_template("login.html", error=error)     

In [ ]:
app.run()

## Python challenge

1. Extend your Fahrenheit Celcius converter website with a landing page. The landing page should have one form with an input field and a submit button. The user types in a Celcius number in the input field. When the Submit button is pressed, the user should see the converted value in Fahrenheit.
    
    **Unsure how to get started?**: Look the above [demo server](https://github.com/UiO-INF3331/UiO-INF3331.github.io/raw/crash-course/lectures/web_programming/web_programming_form.zip) with a working form.



2. **Bonus**: Add a second input field to convert from Fahrenheit to Celcius. 